In [ ]:
# instruciton datasetの生成

In [ ]:
import os

#dataフォルダ内をリセット
os.system("mkdir data")
os.system("rm -rf data/*")

In [ ]:
from datasets import load_dataset
import json

# 自動生成したQAの読み込み

In [ ]:
dataset=load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train")

In [ ]:
print(len(dataset))

In [ ]:
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

In [ ]:
#original template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

#custom template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
answer_template="<SEP>応答<SEP>"


#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。:### 指示::"
#answer_template=":### 応答::"

In [ ]:
dataset[0]

In [ ]:
#ベンチマークに用いるjmt benchと類似しすぎたデータは使わないようにする
import pandas as pd
#!pip install rapidfuzz
from rapidfuzz.process import cdist

jmt_bench_df=pd.read_csv("reference_data/jmtbench.csv")
bench_questions=jmt_bench_df["問い"].tolist()

def check_jmt_similarity(q,bench_questions):
    return 1  #一度チェックして問題ないなら、チェックしない
    scores = cdist([q], bench_questions,workers=1)
    score=max(scores[0])
    return score

In [ ]:
from tqdm import tqdm
score_threshold=4
sim_threshold=80
records=[]

for original_record in tqdm(iter(dataset)):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    if original_record["score"] is None:
        continue
    if int(original_record["score"])<score_threshold:
        continue

    if check_jmt_similarity(q,bench_questions)>sim_threshold:
        print("too similar to jmt bench",q)
        continue

    #if len(a)<10:
    #    print("too short answer",a)
    #    continue

    text=f"{question_template}{q}{answer_template}{a}"
    if a!="":
        records.append(text)



len(records)

In [ ]:
#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds2

In [ ]:
bumpo_records=[]
for original_record in iter(ds2):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    inst=clean_autogen(original_record["instruction"])
    
    text=f"{question_template}{inst}\n{q}{answer_template}{a}"

    bumpo_records.append(text)
print(text)

In [ ]:
import random

def write_jsonl(records,
    output_path="data/gousei.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    random.shuffle(records)

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    with open (output_path,"w", newline='\n') as f:
        temp_lines=lines[:-n_eval][:n_train]
        obj = map(lambda x: x + "\n", temp_lines)
        f.writelines(obj)

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    lines=lines[-n_eval:]
    obj = map(lambda x: x + "\n", lines)
    with open (output_path+".eval","w", newline='\n') as f:
        f.writelines(obj)

n_train=10**7
write_jsonl(records,"data/gousei.jsonl",n_train=n_train)
write_jsonl(bumpo_records,"data/bumpo.jsonl",n_train=n_train)
write_jsonl(records+bumpo_records,"data/gousei_bumpo.jsonl",n_train=n_train)